In [20]:
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
COLS = plotly.colors.DEFAULT_PLOTLY_COLORS

def mass_breakdown_sun_plot(drone_file_path: str, file_formatter=None):
    """
    Returns a figure sunburst plot of the mass breakdown.

    :param drone_file_path: path of data file
    :param file_formatter: the formatter that defines the format of data file. If not provided, default format will
                           be assumed.
    :return: sunburst plot figure
    """
    variables = VariableIO(drone_file_path, file_formatter).read()

    # PROPULSION
    propellers = variables["data:propeller:mass"].value[0] * variables["data:propeller:number"].value[0]
    motors = variables["data:motor:mass"].value[0] * variables["data:propeller:number"].value[0]
    if "data:gearbox:mass" in variables.names():
        gearboxes = variables["data:gearbox:mass"].value[0] * variables["data:propeller:number"].value[0]
    else:
        gearboxes = 0
    ESC = variables["data:ESC:mass"].value[0] * variables["data:propeller:number"].value[0]
    battery = variables["data:battery:mass"].value[0]
    propulsion = propellers + motors + gearboxes + ESC + battery
    
    # STRUCTURE
    frame = variables["data:structure:body:mass"].value[0]
    arms = variables["data:structure:arms:mass"].value[0]
    structure = frame + arms
    
    # PAYLOAD
    payload = variables["specifications:payload:mass:max"].value[0]
    
    # FUEL MISSION (not used yet. May be useful for hydrogen)
    fuel_mission = 0
    
    # MTOW
    MTOW = variables["data:system:MTOW"].value[0]
    # TODO: Deal with this in a more generic manner ?
    if round(MTOW, 6) == round(propulsion + structure + payload + fuel_mission, 6):
        MTOW = propulsion + structure + payload + fuel_mission

        
    # DISPLAYED NAMES AND VALUES
    propellers_str = (
        "Propellers"
        + "<br>"
        + str("{0:.2f}".format(propellers))
        + " [kg] ("
        + str(round(propellers / propulsion * 100, 1))
        + "%)"
    )
    motors_str = (
        "Motors"
        + "<br>"
        + str("{0:.2f}".format(motors))
        + " [kg] ("
        + str(round(motors / propulsion * 100, 1))
        + "%)"
    )
    gearboxes_str = (
        "Gearboxes"
        + "<br>"
        + str("{0:.2f}".format(gearboxes))
        + " [kg] ("
        + str(round(gearboxes / propulsion * 100, 1))
        + "%)"
    )
    ESC_str = (
        "ESC"
        + "<br>"
        + str("{0:.2f}".format(ESC))
        + " [kg] ("
        + str(round(ESC / propulsion * 100, 1))
        + "%)"
    )
    battery_str = (
        "Battery"
        + "<br>"
        + str("{0:.2f}".format(battery))
        + " [kg] ("
        + str(round(battery / propulsion * 100, 1))
        + "%)"
    )
    propulsion_str = (
        "Propulsion"
        + "<br>"
        + str("{0:.2f}".format(propulsion))
        + " [kg] ("
        + str(round(propulsion / MTOW * 100, 1))
        + "%)"
    )
    
    arms_str = (
        "Arms"
        + "<br>"
        + str("{0:.2f}".format(arms))
        + " [kg] ("
        + str(round(arms / structure * 100, 1))
        + "%)"
    )
    frame_str = (
        "Frame"
        + "<br>"
        + str("{0:.2f}".format(frame))
        + " [kg] ("
        + str(round(frame / structure * 100, 1))
        + "%)"
    )
    structure_str = (
        "Structure"
        + "<br>"
        + str("{0:.2f}".format(structure))
        + " [kg] ("
        + str(round(structure / MTOW * 100, 1))
        + "%)"
    )
    
    payload_str = (
        "payload"
        + "<br>"
        + str("{0:.2f}".format(payload))
        + " [kg] ("
        + str(round(payload / MTOW * 100, 1))
        + "%)"
    )
    
    fuel_mission_str = (
        "fuel_mission"
        + "<br>"
        + str("{0:.2f}".format(fuel_mission))
        + " [kg] ("
        + str(round(fuel_mission / MTOW * 100, 1))
        + "%)"
    )
    
    MTOW_str = (
        "MTOW" + "<br>" + str("{0:.2f}".format(MTOW)) + " [kg]"
    )
    
    
    
    # CREATE SUNBURST FIGURE
    fig = go.Figure(
            go.Sunburst(
            labels=[
                MTOW_str,
                payload_str,
                fuel_mission_str,
                propulsion_str,
                structure_str,
                propellers_str,
                motors_str,
                gearboxes_str,
                ESC_str,
                battery_str,
                frame_str,
                arms_str,
            ],
            parents=[
                "",
                MTOW_str,
                MTOW_str,
                MTOW_str,
                MTOW_str,
                propulsion_str,
                propulsion_str,
                propulsion_str,
                propulsion_str,
                propulsion_str,
                structure_str,
                structure_str,
            ],
            values=[
                MTOW, 
                payload, 
                fuel_mission, 
                propulsion,
                structure,
                propellers,
                motors,
                gearboxes,
                ESC,
                battery,
                frame,
                arms,
            ],
            branchvalues="total",
        ),
    )


    fig.update_layout(margin = dict(t=80, l=0, r=0, b=0), title_text="Mass Breakdown", title_x=0.5)

    return fig

In [21]:
import os.path as pth
from fastoad.io import VariableIO

DATA_FOLDER_PATH = '../data'
WORK_FOLDER_PATH = '../workdir'

MDA_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_mda.xml')
MDO_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_mdo.xml')

fig = mass_breakdown_sun_plot(MDO_OUTPUT_FILE)
fig.show()

In [24]:
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
COLS = plotly.colors.DEFAULT_PLOTLY_COLORS

def mass_breakdown_bar_plot(
    drone_file_path: str, name=None, fig=None, file_formatter=None
) -> go.FigureWidget:
    """
    Returns a figure plot of the aircraft mass breakdown using bar plots.
    Different designs can be superposed by providing an existing fig.
    Each design can be provided a name.

    :param aircraft_file_path: path of data file
    :param name: name to give to the trace added to the figure
    :param fig: existing figure to which add the plot
    :param file_formatter: the formatter that defines the format of data file. If not provided, default format will
                           be assumed.
    :return: bar plot figure
    """
    variables = VariableIO(drone_file_path, file_formatter).read()

    # PROPULSION
    propellers = variables["data:propeller:mass"].value[0] * variables["data:propeller:number"].value[0]
    motors = variables["data:motor:mass"].value[0] * variables["data:propeller:number"].value[0]
    if "data:gearbox:mass" in variables.names():
        gearboxes = variables["data:gearbox:mass"].value[0] * variables["data:propeller:number"].value[0]
    else:
        gearboxes = 0
    ESC = variables["data:ESC:mass"].value[0] * variables["data:propeller:number"].value[0]
    battery = variables["data:battery:mass"].value[0]
    propulsion = propellers + motors + gearboxes + ESC + battery
    
    # STRUCTURE
    frame = variables["data:structure:body:mass"].value[0]
    arms = variables["data:structure:arms:mass"].value[0]
    structure = frame + arms
    
    # PAYLOAD
    payload = variables["specifications:payload:mass:max"].value[0]
    
    # FUEL MISSION (not used yet. May be useful for hydrogen)
    fuel_mission = 0
    
    # MTOW
    MTOW = variables["data:system:MTOW"].value[0]
    # TODO: Deal with this in a more generic manner ?
    if round(MTOW, 6) == round(propulsion + structure + payload + fuel_mission, 6):
        MTOW = propulsion + structure + payload + fuel_mission

        
    # DISPLAYED NAMES AND VALUES
    weight_labels = ["MTOW", "Structure", "Propellers", "Motors", "ESC", "Battery", "Gearboxes"]
    weight_values = [MTOW, structure, propellers, motors, ESC, battery, gearboxes]
    
    if fig is None:
        fig = go.Figure()
        
    # Same color for each drone configuration
    i = len(fig.data)
    fig.add_trace(
        go.Bar(name=name, x=weight_labels, y=weight_values, marker_color=COLS[i]),
    )

    fig.update_layout(margin = dict(t=80, l=0, r=0, b=0), title_text="Mass Breakdown", title_x=0.5)
    fig.update_layout(yaxis_title="[kg]")

    return fig

In [25]:
import os.path as pth
from fastoad.io import VariableIO

DATA_FOLDER_PATH = '../data'
WORK_FOLDER_PATH = '../workdir'

MDA_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_mda.xml')
MDO_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_mdo.xml')

fig = mass_breakdown_bar_plot(MDA_OUTPUT_FILE, name='Drone MDA')
fig = mass_breakdown_bar_plot(MDO_OUTPUT_FILE, name='Drone MDO', fig=fig)
fig.show()